In [9]:
import pandas as pd
import numpy as np

1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender

In [10]:
loan_lenders=pd.read_csv('C:/Users/amministratore/Desktop/CsvProgetto/loans_lenders.csv')

In [11]:
#trasformo la stringa in una lista con i nomi dei prestatori 
loan_lenders['lenders']=loan_lenders['lenders'].str.split(',')
#creo un nuovo dataframe
loan_lenders_normalize = pd.DataFrame(columns = ['loan_id', 'lenders'])
#ogni elemento della lista viene trasformato in una riga 
loan_lenders_normalize=loan_lenders.explode('lenders')
loan_lenders_normalize.reset_index(drop= True, inplace=True)
loan_lenders_normalize

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.


In [14]:
loans=pd.read_csv('C:/Users/amministratore/Desktop/CsvProgetto/loans.csv')

In [15]:
#i valori nulli sono NaT e quando viene effettuata la sottrazione il risultato è NaN
loans['disburse_time']=pd.to_datetime(loans['disburse_time'], format = '%Y-%m-%d %H:%M:%S.%f %z')
loans['planned_expiration_time']=pd.to_datetime(loans['planned_expiration_time'], format = '%Y-%m-%d %H:%M:%S.%f %z')
loans['duration']=(loans['planned_expiration_time']-loans['disburse_time']).dt.days
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,7.0,3,2,1,NaN,female,true,irregular,field_partner,53.0
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,8.0,11,2,1,NaN,female,true,monthly,field_partner,96.0
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37.0
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34.0
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57.0


3. Find the lenders that have funded at least twice

In [16]:
#Raggruppo i dati per 'lenders' e successivamente faccio una somma delle volte in cui compare il lender che
#rappresenta il numero di volte in cui ha contribuito ad effettuare il prestito
number_loans=loan_lenders_normalize.groupby('lenders').agg({'lenders': ['count']})
number_loans

,lenders
,count
lenders,
000,39
00000,39
0002,70
00mike00,1
0101craign0101,71
...,...
zzanita,2
zzcyna7269,1


In [17]:
#Rinomino la colonna in quanto questa è diventata indice del df e la funzione di aggregazione fa diventare multindice
#inoltre resetto l'indice
number_loans.columns=['count_loans']
number_loans=number_loans.reset_index()

In [18]:
#conto coloro che hanno contribuito a finanziare almeno due prestiti
more_one_loan=number_loans[number_loans['count_loans']>1]
more_one_loan

,lenders,count_loans
0,000,39
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
...,...,...
1639018,zyrorl,3
1639020,zzaman,11
1639021,zzanita,2
1639024,zzmcfate,56


4. For each country, compute how many loans have involved that country as borrowers

In [19]:
number_of_loans=loans.groupby('country_name').size()
print(number_of_loans)

country_name
Afghanistan        2337
Albania            3075
Armenia           13952
Azerbaijan        10172
Belize              218
                  ...  
Vietnam           21839
Virgin Islands        2
Yemen              4206
Zambia             1277
Zimbabwe           5513
Length: 96, dtype: int64


5. For each country, compute the overall amount of money borrowed.

In [20]:
country_total_loan=loans.groupby('country_name').sum()['loan_amount']
country_total_loan

country_name
Afghanistan        1967950.0
Albania            4307350.0
Armenia           22950475.0
Azerbaijan        14784625.0
Belize              150175.0
                     ...    
Vietnam           24681100.0
Virgin Islands       10000.0
Yemen              3444000.0
Zambia             1978975.0
Zimbabwe           5851875.0
Name: loan_amount, Length: 96, dtype: float64

6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [21]:
total_amount=loans['loan_amount'].sum()
percentage_total_loan=loans.groupby('country_name').sum()['loan_amount']/total_amount*100
print(percentage_total_loan)

country_name
Afghanistan       0.166573
Albania           0.364586
Armenia           1.942589
Azerbaijan        1.251410
Belize            0.012711
                    ...   
Vietnam           2.089074
Virgin Islands    0.000846
Yemen             0.291509
Zambia            0.167506
Zimbabwe          0.495318
Name: loan_amount, Length: 96, dtype: float64


7. Like the three previous points, but split for each year (with respect to disburse time).


In [22]:
loans['year']=loans['disburse_time'].dt.year
loans_of_year=loans.groupby(['country_name','year']).size()
print(loans_of_year)

country_name  year  
Afghanistan   2007.0     408
              2008.0     370
              2009.0     678
              2010.0     632
              2011.0     247
                        ... 
Zimbabwe      2013.0     426
              2014.0    2078
              2015.0     600
              2016.0     808
              2017.0    1079
Length: 748, dtype: int64


In [23]:
total_loans_year=loans.groupby(['country_name','year']).sum()['loan_amount']
print(total_loans_year)

country_name  year  
Afghanistan   2007.0     194975.0
              2008.0     365375.0
              2009.0     585125.0
              2010.0     563350.0
              2011.0     245125.0
                          ...    
Zimbabwe      2013.0     678525.0
              2014.0    1311575.0
              2015.0     723625.0
              2016.0     788600.0
              2017.0    1237600.0
Name: loan_amount, Length: 748, dtype: float64


In [24]:
percentage_loans_year=loans.groupby(['country_name','year']).sum()['loan_amount']/total_amount*100
print(percentage_loans_year)

country_name  year  
Afghanistan   2007.0    0.016503
              2008.0    0.030926
              2009.0    0.049527
              2010.0    0.047683
              2011.0    0.020748
                          ...   
Zimbabwe      2013.0    0.057432
              2014.0    0.111015
              2015.0    0.061250
              2016.0    0.066749
              2017.0    0.104754
Name: loan_amount, Length: 748, dtype: float64


8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


In [25]:
#calcolo la quantità di prestito per ogni lender supponendo che tutti abbiano contribuito in modo uguale
loans['unit_loan_amount']=loans['loan_amount']/loans['num_lenders_total']

In [26]:
#reimposto gli indici
loans.set_index('loan_id',inplace=True)
loan_lenders.set_index('loan_id',inplace=True)
loans.sort_index(inplace=True)
loan_lenders.sort_index(inplace=True)
#join sulle due tabelle consideranno la colonna loan_id
join_df=loans.join(loan_lenders)
join_df

,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,...,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration,year,unit_loan_amount,lenders
loan_id,,,,,,,,,,,,,,,,,,,,,
84,Justine,English,<i>The following description was written by Mo...,NaN,500.0,500.0,funded,Butcher Shop,Food,"Buy bulls, open a butcher shop",...,17,user_favorite,male,true,bullet,field_partner,NaN,2005.0,100.000000,"[ward, michael, brooke]"
85,Geoffrey,English,<i>The following description was written by Mo...,NaN,500.0,500.0,funded,Food Production/Sales,Food,Buying more produce each time for greater profit,...,16,NaN,male,true,bullet,field_partner,NaN,2005.0,166.666667,"[michael, patrick]"
86,Apollo,English,<i>The following description was written by Mo...,NaN,500.0,500.0,funded,Animal Sales,Agriculture,Increase number of goats bought and sold each ...,...,15,NaN,male,true,bullet,field_partner,NaN,2005.0,166.666667,"[michael, METS, erik]"
88,Eunice,English,Eunice received 100 dollar grant from Village ...,NaN,300.0,300.0,funded,Clothing Sales,Clothing,Buy used clothing (mutumbe) of higher quality ...,...,12,NaN,female,true,bullet,field_partner,NaN,2005.0,75.000000,"[michael, susan, ryan]"
89,Rose,English,<i>\r\nThe following description was written b...,NaN,500.0,500.0,funded,Food Production/Sales,Food,Buying and reselling of produce,...,14,NaN,female,true,bullet,field_partner,NaN,2005.0,83.333333,"[hope, michael, dan, ryan]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444072,QAViva,Spanish,Ella tiene 29 anos de edad y habita en casa pr...,A sign seen on an automatic restroom hand drye...,0.0,725.0,fundRaising,Livestock,Agriculture,Translated loan use to english.,...,1,NaN,female,true,monthly,field_partner,30.0,2018.0,inf,NaN
1444082,QAViva,English,To pay for feed and a hutch for the turduckens...,To pay for feed and a hutch for the turduckens...,0.0,50.0,fundRaising,Livestock,Agriculture,To start a turducken farm,...,1,NaN,female,true,monthly,field_partner,30.0,2018.0,inf,NaN
1444083,Spongebob,English,Introducing Series 2. Built-in GPS. Water resi...,Introducing Series 2. Built-in GPS. Water resi...,0.0,50.0,fundRaising,Clothing Sales,Clothing,Apple Watch Series 2,...,1,NaN,male,true,monthly,field_partner,30.0,2018.0,inf,NaN


In [27]:
#creo il nuovo df con i lenders e l'importo unitario prestato e poi normalizzo la tabella
df=join_df[['unit_loan_amount','lenders']]
df.reset_index(inplace=True)


In [28]:
df=df.explode('lenders')

In [29]:
df.head(100)

,loan_id,unit_loan_amount,lenders
0,84,100.000000,ward
0,84,100.000000,michael
0,84,100.000000,brooke
1,85,166.666667,michael
1,85,166.666667,patrick
...,...,...,...
35,124,55.555556,carlhaynes
35,124,55.555556,tim2585
36,125,71.428571,robinsloan
36,125,71.428571,misterajc


In [30]:
#raggruppo per lenders e trovo l'ammontare di prestito per ognuno di loro
final_df= df.groupby('lenders').sum()['unit_loan_amount']
final_df

lenders
 000               1485.309656
 00000             1249.947363
 0002              2201.180463
 00mike00            38.461538
 0101craign0101    2424.088932
                      ...     
zzanita              62.500000
zzcyna7269           44.285714
zzinnia              32.758621
zzmcfate           2033.750197
zzrvmf8538          112.820513
Name: unit_loan_amount, Length: 1639026, dtype: float64

9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [31]:
lenders= pd.read_csv('C:/Users/amministratore/Desktop/CsvProgetto/lenders.csv')

In [32]:
#Calcolo del totale del denaro preso a prestito per ogni paese
amount_money_borrowed=loans.groupby('country_code').sum()['loan_amount']
amount_money_borrowed

country_code
AF     1967950.0
AL     4307350.0
AM    22950475.0
AZ    14784625.0
BA      477250.0
         ...    
XK     3083025.0
YE     3444000.0
ZA     1006525.0
ZM     1978975.0
ZW     5851875.0
Name: loan_amount, Length: 95, dtype: float64

In [33]:
#calcolo della probabilità che una prestito si riferisca ad un determinato paese
prob_amount_money_lent=lenders.country_code.value_counts(normalize=True)

In [34]:
#controllo dove il valore di country_code è nullo
missing = lenders['country_code'].isnull()

In [35]:
#rimpiazzo i valori nulli di country_code con una probabilità uguale a quella calcolata sopra
lenders.loc[missing,'country_code'] = np.random.choice(prob_amount_money_lent.index, size=len(lenders[missing]),p=prob_amount_money_lent.values)

In [36]:
#trasformo in df il risultato del punto precedente dove per ogni prestatore c'è la quantità di denaro prestata
tot_lenders=pd.DataFrame(final_df)
tot_lenders

,unit_loan_amount
lenders,
000,1485.309656
00000,1249.947363
0002,2201.180463
00mike00,38.461538
0101craign0101,2424.088932
...,...
zzanita,62.500000
zzcyna7269,44.285714
zzinnia,32.758621


In [37]:
#mi serve per impostare la colonna su cui verrà fatto il join
lenders.set_index('permanent_name', inplace = True)

In [38]:
#riordino gli indici
lenders.sort_index(inplace=True)
tot_lenders.sort_index(inplace=True)

In [39]:
#join tra il df lenders e tot lenders appena creato
lenders_new=lenders.join(tot_lenders)

In [40]:
#raggruppo per paese e faccio la somma dei prestiti per singolo prestatore
lenders_new=lenders_new.groupby('country_code').sum()['unit_loan_amount']

In [41]:
#trasformo in df il risultato ottenuto
lenders_new=lenders_new.to_frame()
lenders_new

,unit_loan_amount
country_code,
AD,167.579365
AE,113457.546564
AF,6266.861220
AG,31.250000
AI,31.250000
...,...
YE,5295.593159
YT,0.000000
ZA,29977.210463


In [42]:
#rinonimo la colonna 
lenders_new.rename(columns={'unit_loan_amount': 'lent_for_country'}, inplace=True)

In [43]:
#trasformo in df il risultato calcolato all'inizio del punto dove c'è la quantità di denaro presa a prestito per ogni paese
amount_money_borrowed=amount_money_borrowed.to_frame()

In [44]:
result=amount_money_borrowed.join(lenders_new)

In [45]:
result=result.fillna(0)

In [46]:
#differenza tra denaro prestato e preso a prestito per ogni paese
result['diff']=result['lent_for_country']- result['loan_amount']
result.head()

,loan_amount,lent_for_country,diff
country_code,,,
AF,1967950.0,6266.861220,-1.961683e+06
AL,4307350.0,1980.026367,-4.305370e+06
AM,22950475.0,1438.754953,-2.294904e+07
AZ,14784625.0,499.328612,-1.478413e+07
BA,477250.0,5163.056163,-4.720869e+05


10. Which country has the highest ratio between the difference computed at the previous point and the population?


In [65]:
country_stats= pd.read_csv('C:/Users/amministratore/Desktop/CsvProgetto/country_stats.csv')

In [66]:
#calcolo la popolazione per ogni stato
population=country_stats.groupby('country_code').sum()['population']

In [67]:
population=population.to_frame()
population

,population
country_code,
AE,9400145
AF,35530081
AL,2930187
AM,2930450
AO,29784193
...,...
XK,1895250
YE,28250420
ZA,56717156


In [68]:
#join tra il df dei paesi e quello riguardante la differenza tra denaro prestato e preso a prestito
country1=country_stats.join(result,on='country_code')

In [69]:
country1['ratio']=country1['diff']/country1['population']

In [70]:
country1.loc[country1['ratio'].idxmax()]

country_name                               Canada
country_code                                   CA
country_code3                                 CAN
continent                                Americas
region                           Northern America
population                               36624199
population_below_poverty_line                 9.4
hdi                                      0.920284
life_expectancy                            82.224
expected_years_of_schooling                16.325
mean_years_of_schooling                   13.1051
gni                                       42581.9
kiva_country_name                          Canada
loan_amount                                 50000
lent_for_country                        4.829e+06
diff                                    4.779e+06
ratio                                    0.130487
Name: 72, dtype: object

11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?


In [71]:
#calcolo le persone che sono sotto la soglia della povertà
country1['people_below_poverty_line']=(country1['population']*country1['population_below_poverty_line'])/100

In [72]:
#calcolo le persone non povere facendo la differenza tra la popolazione totale e quelle sotto la soglia di povertà
country1['people_above_poverty_line']=country1['population'] - country1['people_below_poverty_line']

In [73]:
country1['ratio1']=country1['diff']/country1['people_above_poverty_line']

In [74]:
country1.loc[country1['ratio1'].idxmax()]

country_name                               Canada
country_code                                   CA
country_code3                                 CAN
continent                                Americas
region                           Northern America
population                               36624199
population_below_poverty_line                 9.4
hdi                                      0.920284
life_expectancy                            82.224
expected_years_of_schooling                16.325
mean_years_of_schooling                   13.1051
gni                                       42581.9
kiva_country_name                          Canada
loan_amount                                 50000
lent_for_country                        4.829e+06
diff                                    4.779e+06
ratio                                    0.130487
people_below_poverty_line             3.44267e+06
people_above_poverty_line             3.31815e+07
ratio1                                   0.144026


12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.


In [78]:
loans=pd.read_csv('C:/Users/amministratore/Desktop/CsvProgetto/loans.csv')

In [79]:
loans['disburse_time']=pd.to_datetime(loans['disburse_time'], format = '%Y-%m-%d %H:%M:%S.%f %z')
loans['planned_expiration_time']=pd.to_datetime(loans['planned_expiration_time'], format = '%Y-%m-%d %H:%M:%S.%f %z')
loans['duration']=(loans['planned_expiration_time']-loans['disburse_time']).dt.days

In [80]:
#aggiungo una colonna in cui verrà messo l'importo loans amount se l'anno dell'esborso e della scadenza del prestito è lo stesso e la duration>0
loans['amount_same_year']= np.where((loans['planned_expiration_time'].dt.year == loans['disburse_time'].dt.year) & (loans['duration']>0), loans['loan_amount'],0)

In [81]:
#specifico l'anno di disburse time ed expiration time in una colonna per facilitare il calcolo successivo
loans['year_disburse']=loans['disburse_time'].dt.year
loans['year_expiration']=loans['planned_expiration_time'].dt.year

In [82]:
#calcolo l'importo prestato per ogni anno per i prestiti che sono stati fatti e rimborsati nello stesso anno
amount_same_year=loans.groupby('year_disburse').sum()['amount_same_year']

In [83]:
#calcolo l'ammontare annuo dei prestiti dove disburse_time>planned_expiration_time
loans['amount_disburse_year']= np.where(loans['duration']<0, loans['loan_amount'],0)

In [84]:
#assegno l'ammontare di quei prestiti con duration<0 alla competenza dell'anno disburse
amount_disburse_year=loans.groupby('year_disburse').sum()['amount_disburse_year']

In [85]:
amount_same_year=amount_same_year.to_frame()

In [86]:
amount_disburse_year=amount_disburse_year.to_frame()

In [87]:
#unisco i due df e faccio la somma per anno dei totali calcolati precedentemente
df_year=amount_same_year.join(amount_disburse_year)
tot_parziale=df_year['amount_same_year']+df_year['amount_disburse_year']

In [88]:
tot_parziale=tot_parziale.to_frame()

In [89]:
tot_parziale.reset_index(inplace=True)

In [91]:
tot_parziale.rename(columns={"year_disburse":"Year",0:"tot_amount"},inplace=True)

In [92]:
tot_parziale

,Year,tot_amount
0,2005.0,0.0
1,2006.0,0.0
2,2007.0,0.0
3,2008.0,0.0
4,2009.0,0.0
5,2010.0,0.0
6,2011.0,0.0
7,2012.0,104060575.0
8,2013.0,99861325.0
9,2014.0,122917400.0


In [93]:
#per i prestiti erogati e rimborsati in anni diversi, calcolo il numero dei giorni del primo anno e dell'ultimo anno
#inoltre aggiungo una colonna in cui è indicato l'anno a cui si fa riferimento
loans['days_first_year']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,365 - (loans['disburse_time'].dt.dayofyear) ,0)
loans['first_year']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['year_disburse'] ,0)
loans['days_last_year']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['planned_expiration_time'].dt.dayofyear ,0)
loans['last_year']=np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['year_expiration'],0)

In [94]:
#per ogni prestito per cui data di erogazione e di scadenza hanno una differenza negli anni >=2, aggiungo una 
#colonna contenente il numero di giorni dell'anno (365) e una colonna in cui viene indicato l'anno a cui si
#fa riferimento. loans.loc[loans['duration'].idxmax()] -> questo comando mi ha permesso di verificare quale fosse
#la duration maggiore e di capire così quanti erano al massimo gli anni intermedi necessari
#anno n+1
loans['days_intermediate_year_1']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 2,365 ,0)
loans['intermediate_year_1']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 2,loans['year_disburse']+1 ,0)
#anno n+2
loans['days_intermediate_year_2']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 3,365 ,0)
loans['intermediate_year_2']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 3,loans['year_disburse']+2 ,0)
#anno n+3
loans['days_intermediate_year_3']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 4,365 ,0)
loans['intermediate_year_3']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 4,loans['year_disburse']+3 ,0)
#anno n+4
loans['days_intermediate_year_4']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 5,365 ,0)
loans['intermediate_year_4']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 5,loans['year_disburse']+4 ,0)

In [95]:
#calcolo il coefficiente di riparto per tutti quei prestiti per cui anno di erogazione e scadenza del prestito
#sono differenti
loans['coeff_riparto']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['loan_amount']/loans['duration'] ,0)

In [96]:
#moltiplico il coefficiente di riparto per il numero di giorni di competenza di ogni anno
loans['amount_first_year']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['coeff_riparto']*loans['days_first_year'] ,0)
loans['amount_intermediate_year_1']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['coeff_riparto']*loans['days_intermediate_year_1'] ,0)
loans['amount_intermediate_year_2']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['coeff_riparto']*loans['days_intermediate_year_2'] ,0)
loans['amount_intermediate_year_3']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['coeff_riparto']*loans['days_intermediate_year_3'] ,0)
loans['amount_intermediate_year_4']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['coeff_riparto']*loans['days_intermediate_year_4'] ,0)
loans['amount_last_year']= np.where((loans['year_expiration'] - loans['year_disburse']) >= 1,loans['coeff_riparto']*loans['days_last_year'] ,0)

In [97]:
#per ogni anno (primo, intermedi e ultimo) raggruppo per anno e faccio la somma dei prestiti
tot_amount_first_year=loans.groupby('first_year').sum()['amount_first_year']
tot_amount_intermediate_year_1=loans.groupby('intermediate_year_1').sum()['amount_intermediate_year_1']
tot_amount_intermediate_year_2=loans.groupby('intermediate_year_2').sum()['amount_intermediate_year_2']
tot_amount_intermediate_year_3=loans.groupby('intermediate_year_3').sum()['amount_intermediate_year_3']
tot_amount_intermediate_year_4=loans.groupby('intermediate_year_4').sum()['amount_intermediate_year_4']
tot_amount_last_year=loans.groupby('last_year').sum()['amount_last_year']

In [98]:
#converto in dataframe ciò che è stato calcolato al punto precedente
tot_amount_first_year= tot_amount_first_year.to_frame()
tot_amount_intermediate_year_1=tot_amount_intermediate_year_1.to_frame()
tot_amount_intermediate_year_2=tot_amount_intermediate_year_2.to_frame()
tot_amount_intermediate_year_3=tot_amount_intermediate_year_3.to_frame()
tot_amount_intermediate_year_4=tot_amount_intermediate_year_4.to_frame()
tot_amount_last_year=tot_amount_last_year.to_frame()

In [99]:
#reset index e rinomino le colonne
tot_amount_first_year.reset_index(inplace=True)
tot_amount_intermediate_year_1.reset_index(inplace=True)
tot_amount_intermediate_year_2.reset_index(inplace=True)
tot_amount_intermediate_year_3.reset_index(inplace=True)
tot_amount_intermediate_year_4.reset_index(inplace=True)
tot_amount_last_year.reset_index(inplace=True)

In [100]:
tot_amount_first_year.rename(columns={"first_year":"Year","amount_first_year":"tot_amount_first_year"},inplace=True)

In [101]:
tot_amount_intermediate_year_1.rename(columns={"intermediate_year_1":"Year","amount_intermediate_year_1":"tot_amount_int_1"},inplace=True)

In [102]:
tot_amount_intermediate_year_2.rename(columns={"intermediate_year_2":"Year","amount_intermediate_year_2":"tot_amount_int_2"},inplace=True)

In [103]:
tot_amount_intermediate_year_3.rename(columns={"intermediate_year_3":"Year","amount_intermediate_year_3":"tot_amount_int_3"},inplace=True)

In [104]:
tot_amount_intermediate_year_4.rename(columns={"intermediate_year_4":"Year","amount_intermediate_year_4":"tot_amount_int_4"},inplace=True)

In [105]:
tot_amount_last_year.rename(columns={"last_year":"Year","amount_last_year":"tot_amount_last_year"},inplace=True)

In [106]:
frames=[tot_amount_first_year, tot_amount_intermediate_year_1, tot_amount_intermediate_year_2, tot_amount_intermediate_year_3, tot_amount_intermediate_year_4,tot_amount_last_year,tot_parziale]

In [107]:
#faccio una concatenazione di tutti i df creati precedentemente
result=pd.concat(frames,sort=True)

In [108]:
#il risultato di questa concat lo raggruppo per anno e faccio la somma 
result=result.groupby('Year').sum()[['tot_amount','tot_amount_first_year','tot_amount_int_1','tot_amount_int_2','tot_amount_int_3','tot_amount_int_4','tot_amount_last_year']]

In [109]:
result

,tot_amount,tot_amount_first_year,tot_amount_int_1,tot_amount_int_2,tot_amount_int_3,tot_amount_int_4,tot_amount_last_year
Year,,,,,,,
0.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2005.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2006.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2007.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2008.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2009.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2010.0,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2011.0,0.0,6.316783e+05,0.000000,0.000000,0.000000,0.000000,0.000000e+00
2012.0,104060575.0,6.866006e+06,6970.144909,0.000000,0.000000,0.000000,2.011036e+06


In [110]:
#faccio una somma per riga, per ogni anno vengono sommati tutti gli importi dei vari df creati in precedenza
result['tot_anno']=result['tot_amount'] + result['tot_amount_int_1']+result['tot_amount_int_2']+result['tot_amount_int_3']+result['tot_amount_int_4']+ result['tot_amount_first_year']+result['tot_amount_last_year']
result['tot_anno'].astype('int64')

Year
0.0               0
2005.0            0
2006.0            0
2007.0            0
2008.0            0
2009.0            0
2010.0            0
2011.0       631678
2012.0    112944586
2013.0    125018930
2014.0    153046963
2015.0    159630978
2016.0    158507154
2017.0    172200270
2018.0      6015073
Name: tot_anno, dtype: int64